In [1]:
load("../Shared/orbits.sage") # Group orbits
load("../Shared/linalg.sage") # Auxiliary linear algebra
load("../Shared/cyclic_covers.sage") # Cyclic covers of function fields
load("../Shared/weil_poly_utils.sage") # Utility functions for Weil polynomials
import itertools, pandas, time
from collections import defaultdict

In [2]:
# 5 elliptic curves over F_2 up to isom in Weierstrass form
ec_coeff = [[0, 0, 1, 1, 1], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1],[0, 0, 1, 1, 0]]
e_lst = []
for coeff in ec_coeff:
    e_lst.append(EllipticCurve(GF(2),coeff))
# p_rank = [0,1,0,1,0] (ordinary = 1, supersingular = 0)
    
e_pc = [] #point counts of EC, E(F_2) = [1,2,3,4,5]
for e in e_lst:
    e_pc.append(tuple(e.count_points(6)))
    
R.<T> = PolynomialRing(QQ) 
e_wp = [T^2 + (-i + 3)*T + 2 for i in range(1,6)]

In [3]:
load("../Shared/pointcounts_p_rank.sage") # data = [ point counts + p-rank ]

In [4]:
# point count check:
# C -> E is of degree 2, so 2*#E(F_2^i) >= #C(F_2^i) 

d = {}
for ecpc in e_pc:
    for cpc in data:
        if all(2*ecpc[i] >= cpc[0][i] for i in range(6)):
            try:
                d[ecpc].append(cpc)
            except:
                d[ecpc] = [cpc]

In [5]:
# resultant criterion: Cor 9.3 of rel class number I
# Deuring-Shafarevich formula: gamma_C - 1 = 2*(gamma_E - 1) + t

#newd[elliptic curve point count ] = [ [curve point count]+[number of geometric ram pts, e(P) > 1] ]

newd = {}
for i in range(5):
    epc = list(d.keys())[i]
    ewp = e_wp[i]
    for cpc in d[epc]:
        u = weil_poly_from_point_count(list(cpc[0]),6)
        if gcd(ewp,u) > 1:
            g = u//ewp
            h1 = ewp.trace_polynomial()[0] #real Weil polys
            h2 = g.trace_polynomial()[0]
            if h1.resultant(h2) %2 == 0: #gcd(d=2,res(h1,h2)) = 2
                if i % 2 == 0 and (cpc[1] <= 4): #supersingular E, cpc[1] = p-rank of cpc 
                    try:
                        newd[epc].append(cpc[0] + [cpc[1]+1])
                    except:
                        newd[epc] = [cpc[0] + [cpc[1]+1]]
                elif i % 2 == 1 and (1<= cpc[1] <= 6): #ordinary E
                    try:
                        newd[epc].append(cpc[0] + [cpc[1]-1])
                    except:
                        newd[epc] = [cpc[0] + [cpc[1]-1]]

In [6]:
d.clear()
data.clear()

In [7]:
[(len(newd[c]),c) for c in newd.keys()]

[(535, (1, 5, 13, 25, 41, 65)),
 (1142, (2, 8, 14, 16, 22, 56)),
 (1719, (3, 9, 9, 9, 33, 81)),
 (781, (4, 8, 4, 16, 44, 56)),
 (1007, (5, 5, 5, 25, 25, 65))]

In [8]:
# Every F_2^i points of E lifts to either an F_2^i ram point or two F_2^2i point of C
# #C(F_2^2i) >= 2*#E(F_2^i) - t

# Every odd degree i-place of C projects to a degree i-place of E
# if t <=2 , #C(F_2^2j-1) == #E(F_2) (mod 2) 

for i in range(5):
    epc = list(newd.keys())[i]
    tmp = newd[epc].copy()
    for cpc in tmp:
        for i in range(3):
            if cpc[2*i + 1] < 2*epc[i] - cpc[-1]:
                try:
                    newd[epc].remove(cpc)
                except:
                    continue
        try:
            if cpc[-1] <= 2 and all( (cpc[2*i] - cpc[0]) % 2 == 1 for i in range(3)):
                newd[epc].remove(cpc)
        except:
            continue    


In [9]:
[(len(newd[c]),c) for c in newd.keys()]

[(531, (1, 5, 13, 25, 41, 65)),
 (1048, (2, 8, 14, 16, 22, 56)),
 (1044, (3, 9, 9, 9, 33, 81)),
 (717, (4, 8, 4, 16, 44, 56)),
 (692, (5, 5, 5, 25, 25, 65))]

In [10]:
# Lemma 6.1 of "New methods for bounding the number of points on curves over finite fields"
for i in range(5):
    epc = list(newd.keys())[i]
    tmp = newd[epc].copy()
    e_place = place_count_from_point_count(list(epc),6)
    for cpc in tmp:
        c_place = place_count_from_point_count(cpc[:-1],6)
        s = sum(k for k in range(0,6,2) if c_place[k] % 2 != 0)
        if s > cpc[-1]:
            try:
                newd[epc].remove(cpc)
            except:
                continue
                
        upp = cpc[-1] - s #t - sum_{odd d, odd a_d} d = upper bound on r_1
        tmp = []
        for r in range(upp+1):
            if (r - cpc[0]) % 2 == 0 and r <= 2*epc[0] - cpc[0] and r >= 2*epc[0] - 2*c_place[1] - cpc[0] and cpc[0] -r >= 0:
                tmp.append(r)
        if len(tmp) == 0:
            try:
                newd[epc].remove(cpc)
            except:
                continue
        else:
            try:
                cpc_new = cpc + [tmp]
                newd[epc].remove(cpc)
                newd[epc].append(cpc_new)
            except:
                continue

In [11]:
[(len(newd[c]),c) for c in newd.keys()]

[(419, (1, 5, 13, 25, 41, 65)),
 (873, (2, 8, 14, 16, 22, 56)),
 (871, (3, 9, 9, 9, 33, 81)),
 (538, (4, 8, 4, 16, 44, 56)),
 (315, (5, 5, 5, 25, 25, 65))]

In [14]:
# Every inertial F_2^i point of E should lift to two F_2^2i points
# so C(F_2^2) >= 2*i1 = 2 * number of F_2 inertial points of E 
# a1 = r1 + 2*s1 = number of degree 1 places on C
# b1 = r1 + s1 + i1 = number of degree 1 places on E

for i in range(5):
    epc = list(newd.keys())[i]
    tmp = newd[epc].copy()
    for cpc in tmp:
        cpc_temp = cpc.copy()
        for r in cpc_temp[-1]:
            s1 = (cpc_temp[0] - r)/2
            i1 = epc[0] - r - s1
            if cpc_temp[1] < 2*i1:
                cpc[-1].remove(r)
            if cpc_temp[6]-r == 1:  # r - r1 = 1 is not possible
                try:
                    cpc[-1].remove(r)
                except:
                    continue
        if len(cpc[-1]) == 0:
            try:
                newd[epc].remove(cpc)
            except:
                continue

In [24]:
[(len(newd[c]),c) for c in newd.keys()]

[(419, (1, 5, 13, 25, 41, 65)),
 (873, (2, 8, 14, 16, 22, 56)),
 (871, (3, 9, 9, 9, 33, 81)),
 (538, (4, 8, 4, 16, 44, 56)),
 (315, (5, 5, 5, 25, 25, 65))]

curves_by_ramification.m only needs ramification data:
- curve_pointcounts.txt contains possible point counts of C over each E
- ram_tuples.txt contains (r1, r-r1), where r1 = number of F_2-rational ramification points on E and r = number of geometric ramification points on E.

In [23]:
for i in range(5):
    ram_lst = {tuple([r,cpc[-2] - r]) for cpc in newd[e_pc[i]] for r in cpc[-1]}
    print(ram_lst)

{(1, 2), (0, 4), (0, 3), (1, 4), (0, 2), (0, 5), (1, 0), (1, 3)}
{(1, 2), (0, 4), (0, 3), (2, 0), (1, 4), (2, 3), (0, 2), (0, 5), (2, 2), (1, 0), (1, 3)}
{(1, 2), (0, 4), (0, 3), (2, 0), (1, 4), (3, 0), (2, 3), (0, 2), (0, 5), (2, 2), (1, 0), (3, 2), (1, 3)}
{(1, 2), (0, 4), (4, 0), (0, 3), (2, 0), (1, 4), (3, 0), (2, 3), (0, 2), (0, 5), (2, 2), (1, 0), (3, 2), (1, 3)}
{(4, 0), (0, 4), (0, 3), (2, 0), (1, 4), (3, 0), (2, 3), (5, 0), (0, 5), (2, 2), (1, 0), (3, 2), (1, 3)}


In [26]:
for i in range(5):
    pc_lst = {tuple(cpc[:6]) for cpc in newd[e_pc[i]]}
    f = open("./data/" + str(i+1) + "/sanity_check/curve_pointcounts.txt","w+")
    for ele in pc_lst:
        f.write(str(list(ele)) + "\n")
    f.close()
    
    ram_lst = {tuple([r,cpc[-2] - r]) for cpc in newd[e_pc[i]] for r in cpc[-1]}
    f = open("./data/" + str(i+1) + "/sanity_check/ram_tuples.txt","w+")
    for ele in ram_lst:
        f.write(str(list(ele)) + "\n")
    f.close()
    
    for ramtup in ram_lst:
        f = open("./data/"+ str(i+1) + "/" + str(ramtup[0]) + "_" + str(ramtup[1]) + ".txt","w+")
        f.close()

In [ ]:
e_lst

In [84]:
for i in e_pc:
    print(place_count_from_point_count(i,6))

[1, 2, 4, 5, 8, 8]
[2, 3, 4, 2, 4, 6]
[3, 3, 2, 0, 6, 11]
[4, 2, 0, 2, 8, 8]
[5, 0, 0, 5, 4, 10]
